<a href="https://colab.research.google.com/github/aditya-r-m/notes/blob/main/2RL_01_exploration_and_control.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Exploration** is the process of learning the environment structure better by trying out actions with potentially low known return, but a high degree of uncertainity.

**Exploitation** is the process of making actions which are known to be optimal to gather maximal cumulative reward.

### Multi-Armed Bandit

We have a set of distributions $\{R_a | a \in A\}$ where $A$ is known set of Actions (Arms). $R_a$ is a distribution on Rewards, given action $a$

The agent selects an action $A_t \in A$ at each timestep $t$ & the environment generates a reward $R_t \sim R_{A_t}$

The goal is to maximize cumulative reward $\sum_{i=1}^t R_i$

We do this by learning a policy, which is a distribution on $A$.

#### Valus & Regret

The action value for action $a$ is the expected reward $q(a) = E[R_t | A_t = a]$

The optimal value is $v_* = \max_{a \in A} q(a) = \max_a E[R_t | A_t = a]$

Regret of an action $a$ is $\Delta_a = v_* - q(a)$ which is zero for optimal action.

The problem of maximizing cumulative reward is identical to minizing total regret $L_t = \sum_{n=1}^t \Delta_{A_n}$ and Good policies will accumulate bounded regret.


#### Algorithms

There are several algorithms to approach the multi-armed bandit problem,

- Greedy
- $\epsilon$-Greedy
- UCB
- Thompson sampling
- Policy gradients

The first three all use action value estimates $Q_t(a) \approx q(a)$


#### Action Values estimation

A simple estimate for $q(a) = E[R_t | A_t = a]$ is the average of sampled rewards,

$
Q_t(a) = \frac{\sum_{n=1}^t I(A_n = a) R_n}{\sum_{n=1}^t I(A_n = a)}
$

where $I(\cdot)$ is the indicator function : $I(True) = 1$ and $I(False) = 0$ and the count for an action $a$ is

$
N_t(a) = \sum_{n=1}^t I(A_n = a)
$

This average can be learnt iteratively as,

$
Q_t(A_t)
= \left[1 - \frac{1}{N_t(A_t)}\right] Q_{t-1}(A_t) + \left[\frac{1}{N_t(A_t)}\right] R_t
= Q_{t-1} + \frac{1}{N_t(A_t)} \left[R_t - Q_{t-1}(A_t)\right]
$,

$
\forall a \neq A_t, Q_t(a) = Q_{t-1}(a)
$

where $N_0(a) = 0$

Note that the step-size $\alpha_t$ which is mulitplied to the error term leads to averaging if it is set to $\frac{1}{N_t(A_t)}$ but other settings are also possible. For example, a constant step-size will lead to tracking instead of averaging.

